# Agents as Tools Pattern Demo

This notebook demonstrates the **Agents as Tools** pattern where:
- Specialized agents are wrapped as callable tools using `@tool` decorator
- An orchestrator agent delegates tasks to appropriate specialists
- Each agent has focused expertise and tools

**Use Case**: Multi-domain customer service assistant

In [1]:
from strands import Agent, tool
from strands.models import BedrockModel

## Step 1: Define Domain-Specific Tools

In [14]:
# Research tools
@tool
def search_knowledge_base(query: str) -> str:
    """Search internal knowledge base"""
    return f"Knowledge base results for '{query}': Found 3 relevant articles"

@tool
def web_search(query: str) -> str:
    """Search the web for information"""
    return f"Web search results for '{query}': Latest information available"

# Product tools
@tool
def search_products(category: str, budget: float) -> str:
    """Search product catalog"""
    return f"Found 5 {category} products under ${budget}"

@tool
def get_product_reviews(product_id: str) -> str:
    """Get product reviews and ratings"""
    return f"Product {product_id}: 4.5 stars, 127 reviews"

# Travel tools
@tool
def search_destinations(preferences: str) -> str:
    """Search travel destinations"""
    return f"Top destinations for {preferences}: Paris, Tokyo, Bali"

@tool
def get_weather(location: str) -> str:
    """Get weather forecast"""
    return f"Weather in {location}: Sunny, 72°F"

## Step 2: Create Specialized Agent Tools

In [15]:
@tool
def research_assistant(query: str) -> str:
    """
    Specialized research assistant for factual information and analysis.
    Use for: research questions, fact-checking, technical information.
    
    Args:
        query: Research question requiring factual information
    
    Returns:
        Detailed research answer with sources
    """
    try:
        agent = Agent(
            system_prompt="""You are a specialized research assistant. 
            Provide factual, well-sourced information. Always cite sources.""",
            model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
            tools=[search_knowledge_base, web_search]
        )
        response = agent(query)
        return str(response)
    except Exception as e:
        return f"Research error: {e}"

In [16]:
@tool
def product_advisor(query: str) -> str:
    """
    Specialized product recommendation assistant.
    Use for: product recommendations, shopping advice, comparisons.
    
    Args:
        query: Product inquiry with preferences and budget
    
    Returns:
        Personalized product recommendations with reasoning
    """
    try:
        agent = Agent(
            system_prompt="""You are a specialized product advisor. 
            Provide personalized recommendations based on user needs and budget.""",
            model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
            tools=[search_products, get_product_reviews]
        )
        response = agent(query)
        return str(response)
    except Exception as e:
        return f"Product advisor error: {e}"

In [17]:
@tool
def travel_planner(query: str) -> str:
    """
    Specialized travel planning assistant.
    Use for: travel itineraries, destination advice, trip planning.
    
    Args:
        query: Travel planning request with destination and preferences
    
    Returns:
        Detailed travel itinerary or destination advice
    """
    try:
        agent = Agent(
            system_prompt="""You are a specialized travel planner. 
            Create detailed itineraries based on preferences and constraints.""",
            model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
            tools=[search_destinations, get_weather]
        )
        response = agent(query)
        return str(response)
    except Exception as e:
        return f"Travel planner error: {e}"

## Step 3: Create Orchestrator Agent

In [22]:
ORCHESTRATOR_PROMPT = """
You are an intelligent orchestrator that routes queries to specialized agents:

- research_assistant: For research questions, factual information, technical queries
- product_advisor: For product recommendations, shopping advice, comparisons
- travel_planner: For travel planning, itineraries, destination advice

For simple questions not requiring specialized knowledge, answer directly.
For complex queries, you may call multiple specialists and synthesize their responses.

Always select the most appropriate specialist(s) based on the user's query.
"""

orchestrator = Agent(
    system_prompt=ORCHESTRATOR_PROMPT,
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
    tools=[research_assistant, product_advisor, travel_planner]
)

## Step 4: Test Single-Domain Queries

In [23]:
# Research query
response = orchestrator("What are the latest developments in quantum computing?")
print("/n")
print("="*180)
print("=== Research Query ===")
print(response)

<thinking> The user is asking for the latest developments in quantum computing, which is a technical and research-oriented question. The most appropriate tool to use here would be the research_assistant since it specializes in providing factual information and analysis. I will use the research_assistant tool to gather the latest developments in quantum computing.</thinking>

Tool #1: research_assistant
<thinking> To provide the latest developments in quantum computing, I should first search the internal knowledge base for recent updates and then use the web search tool to find the most recent articles or news pieces. This will give me a comprehensive view of the current state of quantum computing.</thinking>


Tool #1: search_knowledge_base

Tool #2: web_search
<thinking> The internal knowledge base has provided three relevant articles on the latest developments in quantum computing. The web search tool indicates that the latest information is available. I should proceed to review the 

In [24]:
# Product query
response = orchestrator("I need a laptop under $1000 for software development")
print("/n")
print("="*180)
print("=== Product Query ===")
print(response)

<thinking> The user is looking for a laptop under $1000 for software development. This is a product recommendation request, which falls within the expertise of the product_advisor. I will use the product_advisor tool to provide a recommendation for a laptop suitable for software development.</thinking> 
Tool #2: product_advisor
<thinking> To recommend a laptop for software development under $1000, I need to search for laptops in that budget range that are suitable for this purpose. I will use the "search_products" tool to find suitable laptops in the "computers" category with a budget of $1000. After finding potential options, I will look for reviews to determine which one is the best for software development.</thinking>


Tool #1: search_products
<thinking> I found 5 laptops under $1000 that could be suitable for software development. Next, I need to review these laptops to identify the best option. I will use the "get_product_reviews" tool to retrieve reviews for each of these laptop

In [25]:
# Travel query
response = orchestrator("Plan a 5-day trip to Japan in spring")
print("/n")
print("="*180)
print("=== Travel Query ===")
print(response)

Planning a 5-day trip to Japan in the spring is a wonderful idea, as it allows you to experience the breathtaking cherry blossom season (hanami). Here’s a detailed itinerary to help you make the most of your trip:

### Day 1: Arrival in Tokyo
**Morning:**
- Arrive at Narita or Haneda Airport in Tokyo.
- Take the Narita Express or Airport Limousine Bus to your hotel in central Tokyo.

**Afternoon:**
- Check into your hotel and freshen up.
- Head out to explore the vibrant neighborhood of Shinjuku. Visit the famous Golden Gai and enjoy some local street food.

**Evening:**
- Dinner at a traditional izakaya (Japanese pub) in Shinjuku.
- Stroll through the illuminated streets of Shinjuku and enjoy the city's nightscape.

### Day 2: Tokyo Exploration
**Morning:**
- Visit the historic Senso-ji Temple in Asakusa. Don’t forget to check out the Nakamise Shopping Street for some traditional snacks and souvenirs.
- Explore Ueno Park and Ueno Zoo.

**Afternoon:**
- Lunch in Ueno, perhaps at one of

## Step 5: Test Multi-Domain Query

The orchestrator should intelligently call multiple specialists

In [26]:
# Complex query requiring multiple specialists
complex_query = """
I'm planning a hiking trip to Patagonia next month. 
I need hiking boots suitable for that terrain and weather.
What should I know about the region and what gear do you recommend?
"""

response = orchestrator(complex_query)
print("/n")
print("="*180)
print("=== Multi-Domain Query ===")
print(response)

Patagonia, renowned for its dramatic landscapes and diverse ecosystems, presents both a thrilling and challenging hiking experience. Here's what you should know about the region and what gear is recommended for your trip:

### Understanding the Region:
1. **Terrain:**
   - **Mountains:** The Andes dominate the western part of Patagonia, with peaks that can exceed 4,000 meters (13,000 feet).
   - **Glacial Areas:** Expect glaciers, rocky paths, and moraines.
   - **Steppes and Lakes:** Eastern Patagonia features more open plains and numerous lakes, with some areas resembling pampas.
   - **Wildlife:** The region is home to diverse wildlife, including guanacos, Andean condors, and various bird species.

2. **Weather:**
   - **Temperature:** Can vary greatly; expect cold temperatures, especially at higher altitudes and during winter months.
   - **Precipitation:** Can be heavy in the form of rain or snow, particularly in the west.
   - **Wind:** Patagonia is known for its strong winds, es

## Key Benefits Demonstrated

1. **Separation of Concerns**: Each specialist focuses on its domain
2. **Hierarchical Delegation**: Orchestrator routes to appropriate specialist
3. **Modularity**: Easy to add/remove specialists without affecting others
4. **Optimized Performance**: Each agent has tailored prompts and tools
5. **Multi-Domain Handling**: Complex queries automatically use multiple specialists